# Resume Chatbot

## Web Analytics Final project

This is part of the final project on 620 Web Analytics course. Objective of this project is to create a resume chatbot for Shyam BV. This will provide most of the information about me.

## Introduction

In this notebook, we will build a deep neural network with that functions as part of an end-to-end machine translation pipeline which also uses text processing libraries. A question will be asked, an answer will be provided by bot. Also bot's ability to respond to new questions.

- **Dataset Creation** - Here we will create a dataset with various text inputs
- **Preprocess** - You'll convert text to sequence of integers with various preprocessing.
- **Models** - A Deep neural network model will be created. By creating a word embedding, this model will be able to learn new words.
- **Prediction** - Run the model and generate responses from bot.

## Dataset

To perform this deep learning chatbot, we need to train the model with lots of data. Unfortunately my resume is not in format of QA(question answering type). So we need to generate lot of data with some existing data for this model.
 
Here is a sample data which [I found](
https://raw.githubusercontent.com/bvshyam/make_yourself_a_bot/master/workspace-watson.json). This will be me base to create a dataset.

Over the period, due to many interactions, we will gather more data and our bot will get better over time.

## Dataset creation

As mentioned earlier, need create the initial dataset. That will be performed in  data_creation.ipynb

## Neural Network

As the model will be created using deep neural networks, we will be using Keras library with Tensorflow backend.

In [3]:
from keras.layers import GRU, Input, Dense, TimeDistributed, SimpleRNN, LSTM
from keras.models import Model
from keras.models import Sequential
from keras.layers import Activation
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy
from keras.callbacks import ModelCheckpoint
from keras.layers import Dropout
from keras.layers import Dropout, Bidirectional, RepeatVector
from keras.layers.embeddings import Embedding
import collections
import pandas as pd
import nltk

from keras.preprocessing.text import Tokenizer,text_to_word_sequence
import numpy as np
from keras.preprocessing.sequence import pad_sequences
import numpy as np
from keras.preprocessing.sequence import pad_sequences
from keras.models import load_model
from autocorrect import spell
from scipy.spatial import distance
import time

Using TensorFlow backend.


A dataset has been created with dataset_creation.ipynb and some manual effort to correct it.Now we will load it and perform further pre-processing.

In [4]:
#Create a pandas dataframe for the input data

df = pd.read_csv('./data/final_qa_data.csv',sep=',')
df.head()

,Question,Answer
0,Tell me 5 positive things about you,Tricky question detecting. Waiting for next qu...
1,Tell me your strengths,Tricky question detecting. Waiting for next qu...
2,Tell us Unique Selling Points,Tricky question detecting. Waiting for next qu...
3,What are you good at ?,Tricky question detecting. Waiting for next qu...
4,What are your professional strengths ?,Tricky question detecting. Waiting for next qu...


In [5]:
# Loading dataset to numpy array

question_sentences = df.Question.values
answer_sentences = df.Answer.values

print('Dataset Loaded')

Dataset Loaded


### Files

In each line, the first part is question and then the next part is answers. Lets see sample.

In [7]:
for sample_i in range(2):
    print('Question Line {}:  {}'.format(sample_i + 1, question_sentences[sample_i]))
    print('Answer Line {}:  {}'.format(sample_i + 1, answer_sentences[sample_i]))

Question Line 1:  Tell me 5 positive things about you
Answer Line 1:  Tricky question detecting. Waiting for next question...
Question Line 2:  Tell me your strengths
Answer Line 2:  Tricky question detecting. Waiting for next question...


From looking at the sentences, we need to perform some pre-processing. Below are some of them.

- Complete string lower case. Parent vocab has only lower case characters.
- Separate symbols and text. Else it will be considered as unique word.
- Preserve the line ending.
- Also replace some of the junk characters.
- Replace won't, wouldn't etc into proper words.


### Vocabulary

Also need to build a word vocablary for the questions and answers. This will be our knowledge base. All the new questions will be stored and the vocabulary will be updated.

In [8]:
def preprocess_cleaning(texts):
    
    text_cleaned = []
    
#    for text in texts:
#        for raw_word in text_to_word_sequence(text,filters='', lower=True):
            #
#           l1 = ['won’t','won\'t','wouldn’t','wouldn\'t','’m', '’re', '’ve', '’ll', '’s','’d', 'n’t', '\'m', '\'re', '\'ve', '\'ll', '\'s', '\'d', 'can\'t', 'n\'t', 'B: ', 'A: ', ',', ';', '.', '?', '!', ':', '. ?', ',   .', '. ,', 'EOS', 'BOS', 'eos', 'bos']
#            l2 = ['will not','will not','would not','would not',' am', ' are', ' have', ' will', ' is', ' had', ' not', ' am', ' are', ' have', ' will', ' is', ' had', 'can not', ' not', '', '', ' ,', ' ;', ' .', ' ?', ' !', ' :', '? ', '.', ',', '', '', '', '']

#            raw_word = raw_word.lower()

#            for j, term in enumerate(l1):
#                raw_word = raw_word.replace(term,l2[j])

#            for j in range(30):
#                raw_word = raw_word.replace('. .', '')
#                raw_word = raw_word.replace('.  .', '')
#                raw_word = raw_word.replace('..', '')
#                raw_word = raw_word.replace('...', '')

#            for j in range(5):
#                raw_word = raw_word.replace('  ', ' ')
                
#            text_cleaned.append(raw_word)
            
        #text_cleaned.append(raw_word)
        #print(text_cleaned)
        #Convert to lower and tokenize
        
        #text_cleaned.append(' '.join(text_to_word_sequence(raw_word,filters='', lower=True)))
        
    text_cleaned = [' '.join(nltk.tokenize.word_tokenize(word.lower(),preserve_line=True)) for word in texts]
        
        
        
    return text_cleaned
    

In [9]:
#Clean and store in the same variables.

question_sentences = preprocess_cleaning(question_sentences)
answer_sentences = preprocess_cleaning(answer_sentences)


Nex we will analyze the overall dataset and draw some inference out of it.

In [8]:

question_words_counter = collections.Counter([word for sentence in question_sentences for word in sentence.split()])
answer_words_counter = collections.Counter([word for sentence in answer_sentences for word in sentence.split()])

print('{} words in questions.'.format(len([word for sentence in question_sentences for word in sentence.split()])))
print('{} unique English words.'.format(len(question_words_counter)))
print('10 Most common words in the questions dataset:')
print('"' + '" "'.join(list(zip(*question_words_counter.most_common(10)))[0]) + '"')
print()
print('{} words in answers.'.format(len([word for sentence in answer_sentences for word in sentence.split()])))
print('{} unique English words.'.format(len(answer_words_counter)))
print('10 Most common words in the answers dataset:')
print('"' + '" "'.join(list(zip(*answer_words_counter.most_common(10)))[0]) + '"')

1067 words in questions.
330 unique English words.
10 Most common words in the questions dataset:
"you" "?" "are" "your" "do" "What" "what" "me" "about" "How"

2876 words in answers.
319 unique English words.
10 Most common words in the answers dataset:
"I" "to" "a" "you" "for" "my" "and" "I'm" "is" "me"


In [9]:
#Most common top 20 word.

question_words_counter.most_common(20)

[('you', 101),
 ('?', 73),
 ('are', 34),
 ('your', 31),
 ('do', 31),
 ('What', 26),
 ('what', 26),
 ('me', 17),
 ('about', 17),
 ('How', 16),
 ('to', 16),
 ('is', 15),
 ('a', 14),
 ('I', 13),
 ('the', 12),
 ('how', 11),
 ('work', 11),
 ('can', 10),
 ('have', 10),
 ('tell', 9)]

## Preprocess

As a next step we will perform below ones.
1. Tokenize the words into ids
2. Add padding to make all the sequences the same length.


### Tokenize (IMPLEMENTATION)
For a neural network to predict on text data, it first has to be turned into data it can understand. 

A word level model uses word ids that generate text predictions for each word. Also we will use the pre-trained word embedding called as [Glove](https://nlp.stanford.edu/projects/glove/)

Turn each sentence into a sequence of words ids using Keras's [`Tokenizer`](https://keras.io/preprocessing/text/#tokenizer) function. Using this function we will tokenize `question_sentences` and `answer_sentences` in the cell below.


In [14]:
def tokenize(x):
    """
    Tokenize x
    param x: List of sentences/strings to be tokenized
    return: Tuple of (tokenized x data, tokenizer used to tokenize x)
    """
    # convert to nltk tokenizer to preserve the symbols and tokenize
    x = [' '.join(nltk.tokenize.word_tokenize(word.lower(),preserve_line=True)) for word in x]

    #Use tokenizer from Keras
    tokenizer = Tokenizer(num_words=None, filters="", lower=True, split=" ")
    tokenizer.fit_on_texts(x)
    return tokenizer.texts_to_sequences(x), tokenizer


# Tokenize sample output

text_sentences = question_sentences[:3]
text_tokenized, text_tokenizer = tokenize(text_sentences)
print(text_sentences,"\n")
print(text_tokenizer.word_index)
print()

for sample_i, (sent, token_sent) in enumerate(zip(text_sentences, text_tokenized)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(sent))
    print('  Output: {}'.format(token_sent))

['Tell me 5 positive things about you' 'Tell me your strengths'
 'Tell us Unique Selling Points'] 

{'tell': 1, 'me': 2, '5': 3, 'positive': 4, 'things': 5, 'about': 6, 'you': 7, 'your': 8, 'strengths': 9, 'us': 10, 'unique': 11, 'selling': 12, 'points': 13}

Sequence 1 in x
  Input:  Tell me 5 positive things about you
  Output: [1, 2, 3, 4, 5, 6, 7]
Sequence 2 in x
  Input:  Tell me your strengths
  Output: [1, 2, 8, 9]
Sequence 3 in x
  Input:  Tell us Unique Selling Points
  Output: [1, 10, 11, 12, 13]


### Padding

When batching the sequence of word ids together, each sequence needs to be the same length.  Since sentences are dynamic in length, we can add padding to the end of the sequences to make them the same length.

Make sure all the English sequences have the same length and all the French sequences have the same length by adding padding to the **end** of each sequence using Keras's [`pad_sequences`](https://keras.io/preprocessing/sequence/#pad_sequences) function.

In [11]:
def pad(x, length=None):
    """
    Pad x
    param x: List of sequences.
    param length: Length to pad the sequence to.  If None, use length of longest sequence in x.
    return: Padded numpy array of sequences
    """
    if length==None:
        maxLenX = 0
        for sequence in x:
            if len(sequence) > maxLenX:
                maxLenX = len(sequence)
        
        padded = pad_sequences(sequences=x,maxlen=maxLenX, dtype='int32', padding='post', truncating='post', value=0)

    else:
        padded = pad_sequences(sequences=x,maxlen=length, dtype='int32', padding='post', truncating='post', value=0)
    return padded


# Pad Tokenized output
test_pad = pad(text_tokenized)
for sample_i, (token_sent, pad_sent) in enumerate(zip(text_tokenized, test_pad)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(np.array(token_sent)))
    print('  Output: {}'.format(pad_sent))

Sequence 1 in x
  Input:  [1 2 3 4 5 6 7]
  Output: [1 2 3 4 5 6 7]
Sequence 2 in x
  Input:  [1 2 8 9]
  Output: [1 2 8 9 0 0 0]
Sequence 3 in x
  Input:  [ 1 10 11 12 13]
  Output: [ 1 10 11 12 13  0  0]


### Preprocess Pipeline

As a next step we will call the above functions for the questions and answers and store those tokenizer and outputs for further usage.

In [16]:
def preprocess(x, y):
    """
    Preprocess x and y
    :param x: Feature List of sentences
    :param y: Feature output List of sentences
    :return: Tuple of (Preprocessed x, Preprocessed y, x tokenizer, y tokenizer)
    """
    preprocess_x, x_tk = tokenize(x)
    preprocess_y, y_tk = tokenize(y)

    preprocess_x = pad(preprocess_x)
    preprocess_y = pad(preprocess_y)
    
    print(preprocess_y.shape)
    
    # Keras's sparse_categorical_crossentropy function requires the labels to be in 3 dimensions
    preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)

    return preprocess_x, preprocess_y, x_tk, y_tk

preproc_question_sentences, preproc_answer_sentences, question_tokenizer, answer_tokenizer =\
    preprocess(question_sentences, answer_sentences)

print('Data Preprocessed')


(261, 39)
Data Preprocessed


In [22]:
preproc_question_sentences

array([[ 10,  11,  74, ...,   0,   0,   0],
       [ 10,  11,   6, ...,   0,   0,   0],
       [ 10,  38, 134, ...,   0,   0,   0],
       ..., 
       [  3,   9,  33, ...,   0,   0,   0],
       [  3,   9,  33, ...,   0,   0,   0],
       [  3,   9,  33, ...,   0,   0,   0]])

## Glove Embedding 

As a next step we will try Glove word embedding. Here we want to convert the preprocessed padded word sentences to a word vector with dimentions. By performing this, we know the relationship between the words. 

For this project, we have used a simple word embedding which is of 100 dimensions. Means each word is converted to a 100 dimension vector. That will be one of the input to Embedding layer.

We will assign the word embedding vector to each vocablary in the questions layer. 

In [23]:
#word embeddings from glove

embeddings_index = dict()

with open('./data/glove.6B.100d.txt',encoding='utf8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word]= coefs

print('loaded %s word vectors' % len(embeddings_index))

loaded 400000 word vectors


In [24]:
#Create an encoder matrix with all the vocablary in questions.

num_question_tokens = len(question_tokenizer.word_index)+1

#Create a dummy matrix to store all the word embeddings
encoder_embedding_matrix = np.zeros((num_question_tokens, 100))

# List to find nearest words for unknown words in current vocab list
unknown_helper_list = []

for word,i in question_tokenizer.word_index.items():
    
    embedding_vector = embeddings_index.get(word)
    unknown_helper_list.append(word)

    if embedding_vector is not None:
        encoder_embedding_matrix[i] = embedding_vector
        
#Final matrix is the collection of vectors for the complete questions vocab

## Model Implementation

As a next step, we will build a deep neural network model. This will be a series of different layers. We will specify some hyper parameters and then use it to train our model.

Below are the different layers of deep neural network

1. Embedding layer which acts as input layer and gets input from embeddings vector.
2. Create a Bidirectional LSTM layer.
3. Add some random dropout neurons. Repeat Step 2 and 3.
4. Create an output layer with sigmoid activation function.
5. Finally compile the model with optimizer and accuracy metrics.
6. Fit the model with training questions and answers.

In [25]:
#Learning rate for optimizer
learning_rate = 0.01
# Hidden layers for LSTM layers
hidden_dim = 256
# Word embedding vector length
embedding_vector_length = 100
# Number of epochs for complete training
epochs =1000


In [34]:

# Reshaping the questions input to work with a basic RNN with output shape
tmp_x = pad(preproc_question_sentences, preproc_answer_sentences.shape[1])
tmp_x = tmp_x.reshape((-1, preproc_answer_sentences.shape[-2]))


In [19]:
#Assign to the variables
input_shape = tmp_x.shape
output_sequence_length = preproc_answer_sentences.shape[1]
question_vocab_size = len(question_tokenizer.word_index)+1
answer_vocab_size= len(answer_tokenizer.word_index)+1


In [23]:
#Variation - 3 Hidden LSTM Layer and 2 Dense ouput layer with higher Dropout Rate

model = Sequential()
model.add(Embedding(question_vocab_size, embedding_vector_length,weights=[encoder_embedding_matrix], trainable= False,
                    input_shape= input_shape[1:]))
model.add(Bidirectional(LSTM(hidden_dim, return_sequences=True)))
model.add(Dropout(0.2))
model.add(Bidirectional(LSTM(hidden_dim, return_sequences=True)))
model.add(Dropout(0.2))
model.add(Bidirectional(LSTM(hidden_dim, return_sequences=True)))
model.add(Dropout(0.2))
model.add(Dense(answer_vocab_size))
model.add(Activation('sigmoid'))
model.add(Dropout(0.5))
model.add(Dense(answer_vocab_size))
model.add(Activation('softmax'))
model.compile(loss=sparse_categorical_crossentropy,
              optimizer=Adam(learning_rate),
              metrics=['accuracy'])
print(model.summary())


model.fit(tmp_x, preproc_answer_sentences, batch_size=128, epochs=epochs, validation_split=0.2)



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 39, 100)           28900     
_________________________________________________________________
bidirectional_1 (Bidirection (None, 39, 512)           731136    
_________________________________________________________________
dropout_1 (Dropout)          (None, 39, 512)           0         
_________________________________________________________________
bidirectional_2 (Bidirection (None, 39, 512)           1574912   
_________________________________________________________________
dropout_2 (Dropout)          (None, 39, 512)           0         
_________________________________________________________________
bidirectional_3 (Bidirection (None, 39, 512)           1574912   
_________________________________________________________________
dropout_3 (Dropout)          (None, 39, 512)           0         
__________

In [35]:
#Save the model for future purposes
model.save('./saved_models/resume_chatbot_save_'+str(epochs)+'.hdf5')

### Ids Back to Text

The neural network will be translating the input to words ids, which isn't the final form we want.  We want the answers for that question.  Below functions `logits_to_text` will bridge the gab between the logits from the neural network to the answers. 


In [38]:
def logits_to_text(logits, tokenizer):
    """
    Turn logits from a neural network into text using the tokenizer
    :param logits: Logits from a neural network
    :param tokenizer: Keras Tokenizer fit on the labels
    :return: String that represents the text of the logits
    """
    #index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    #index_to_words[0] = '<PAD>'

    #return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])
    
    return False
print('`logits_to_text` function loaded.')

`logits_to_text` function loaded.


In [39]:
def similar_word(unknown_word_dim):
    """
    To find the similar if a typed word is not available in questions vocab. 
    Here we are finding the nearest word using euclidean distance 
    and perform the approximate word  which is similar to it.
    :param unknown_word_dim: unknown word entered in the text
    """
    all_distance = []

    for known_word in encoder_embedding_matrix:
        all_distance.append(distance.euclidean(unknown_word_dim,known_word))
    
    #Get the minimum distance using argsort
    return(unknown_helper_list[np.array(all_distance).argsort()[:1][0]])

In [45]:
def preprocess_test(raw_word, question_tokenizer):
    """
    Preprocess the text which is entered by user. We need to remove and clean 
    the text before we predict the answer.
    :param raw_word: Raw sentence entered by the user.
    :param question_tokenizer: Question tokenizer for vocab
    """
    
    # Cleaning the text
    l1 = ['won’t','won\'t','wouldn’t','wouldn\'t','’m', '’re', '’ve', '’ll', '’s','’d', 'n’t', '\'m', '\'re', '\'ve', '\'ll', '\'s', '\'d', 'can\'t', 'n\'t', 'B: ', 'A: ', ',', ';', '.', '?', '!', ':', '. ?', ',   .', '. ,', 'EOS', 'BOS', 'eos', 'bos']
    l2 = ['will not','will not','would not','would not',' am', ' are', ' have', ' will', ' is', ' had', ' not', ' am', ' are', ' have', ' will', ' is', ' had', 'can not', ' not', '', '', ' ,', ' ;', ' .', ' ?', ' !', ' :', '? ', '.', ',', '', '', '', '']

    raw_word = raw_word.lower()

    for j, term in enumerate(l1):
        raw_word = raw_word.replace(term,l2[j])
       
    for j in range(30):
        raw_word = raw_word.replace('. .', '')
        raw_word = raw_word.replace('.  .', '')
        raw_word = raw_word.replace('..', '')
        raw_word = raw_word.replace('...', '')
        
    for j in range(5):
        raw_word = raw_word.replace('  ', ' ')
 
    #Spell checker and call similar words function

    final_corrected_words  = []
    
    for text in nltk.tokenize.word_tokenize(raw_word.lower(),preserve_line=True):
        text = text.lower()
        #Spell checker changes the symbols. So passing only strings.
        if text not in '!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n':
            #Spell checker
            text = spell(text).lower()
            
            # Finding unknown similar words from the vocab
            if text not in question_tokenizer.word_index.keys():
                
                try:
                    final_corrected_words.append(similar_word(embeddings_index[text]))
                except:
                    final_corrected_words.append('?')
            else:
                final_corrected_words.append(text)
        else:
           
            if text not in question_tokenizer.word_index.keys():
                final_corrected_words.append(similar_word(embeddings_index[text]))
            else:
                final_corrected_words.append(text)
            
    #print(' '.join(final_corrected_words))
    return ' '.join(final_corrected_words)


In [49]:


que = ''
last_query  = ' '
last_last_query = ''
text = ' '
last_text = ''
name_of_computer = 'Jammy AI'

# Open files to save the conversation for further training:

name = time.strftime("%Y%m%d-%H%M%S")
qf = open('./session_data/'+name+'.txt', 'w')


def final_predictions(model, x, y, x_tk, y_tk):
    """
    Gets predictions using the final model
    :param x: Preprocessed English data
    :param y: Preprocessed French data
    :param x_tk: Questions tokenizer
    :param y_tk: Answers tokenizer
    """
    
    ## Create a answer dictionary
    y_id_to_word = {value: key for key, value in y_tk.word_index.items()}
    y_id_to_word[0] = '<PAD>'
    

    print('Chatbot: Hi ! please type your name.\n')
    name = input('user: ')
    print('Chatbot: hi , ' + name +' ! My name is ' + name_of_computer + '.\n') 

    
    while(True):
        
        que = input()
        
        qf.write("Question typed:" + que + '\n')
        
        if que =='exit':
            break
        else:
            #Preprocess the text
            que = preprocess_test(que,x_tk)
            #print(que)
        sentence =que
        
        qf.write("Question interpreted:" + que + '\n')
        
        #sentence = [x_tk.word_index[word] for word in text_to_word_sequence(sentence,filters='')]
        
        sentence = [x_tk.word_index[word] for word in nltk.tokenize.word_tokenize(sentence.lower())]
        
        #Convert to padded sequence
        sentence = pad_sequences([sentence], maxlen=x.shape[-1], padding='post')
        sentences = np.array([sentence[0], x[0]])
        
        #print(sentences.shape)
        
        tmp_sentences = pad(sentences, y.shape[1])
        tmp_sentences = tmp_sentences.reshape((-1, y.shape[-2]))

        #print(tmp_sentences)
        predictions = model.predict(tmp_sentences, len(tmp_sentences))

        #print('Sample 1:')
        prediction_text = ' '.join([y_id_to_word[np.argmax(x)] for x in predictions[0]]).replace('<PAD>','')
        
        qf.write("Answer by bot:" + prediction_text + '\n')
        print(prediction_text+'\n')

    qf.close()

final_predictions(loaded_model,preproc_question_sentences, preproc_answer_sentences, question_tokenizer, answer_tokenizer)

Chatbot: Hi ! please type your name.

user: SHyam
Chatbot: hi , SHyam ! My name is Jammy AI.

what can you do for me?
think of me as shyam at an ask me `` interview '' related questions. what books i read , what have i doing lately or to tell you a joke .        

tell me a joke
ok.. here it is. roses are \ # ff0000 / violets are \ # 0000ff / all my base / belong to you .               

exit


In [30]:
qf.close()

In [42]:
from keras.models import load_model

#loaded_model = load_model('saved_models/final_rnn_model6.hdf5')
loaded_model = load_model('saved_models/resume_chatbot_save_2000.hdf5')

loaded_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 39, 100)           28900     
_________________________________________________________________
bidirectional_4 (Bidirection (None, 39, 512)           731136    
_________________________________________________________________
dropout_5 (Dropout)          (None, 39, 512)           0         
_________________________________________________________________
bidirectional_5 (Bidirection (None, 39, 512)           1574912   
_________________________________________________________________
dropout_6 (Dropout)          (None, 39, 512)           0         
_________________________________________________________________
bidirectional_6 (Bidirection (None, 39, 512)           1574912   
_________________________________________________________________
dropout_7 (Dropout)          (None, 39, 512)           0         
__________

In [ ]:
#Variation - 3 Hidden LSTM Layer and 2 Dense ouput layer with higher Dropout Rate

# Reshaping the input to work with a basic RNN
tmp_x = pad(preproc_question_sentences, preproc_answer_sentences.shape[1])
tmp_x = tmp_x.reshape((-1, preproc_answer_sentences.shape[-2]))


input_shape =tmp_x.shape
output_sequence_length =preproc_answer_sentences.shape[1]
question_vocab_size =len(question_tokenizer.word_index)+1
answer_vocab_size=len(answer_tokenizer.word_index)+1

model = Sequential()
model.add(Embedding(question_vocab_size, embedding_vector_length,weights=[encoder_embedding_matrix], trainable= False,
                    input_shape= input_shape[1:]))
model.add(Bidirectional(LSTM(hidden_dim, return_sequences=True)))
model.add(Dropout(0.2))
model.add(Bidirectional(LSTM(hidden_dim, return_sequences=True)))
model.add(Dropout(0.2))
model.add(Bidirectional(LSTM(hidden_dim, return_sequences=True)))
model.add(Dropout(0.2))
model.add(Dense(answer_vocab_size))
model.add(Activation('sigmoid'))
model.add(Dropout(0.5))
model.add(Dense(answer_vocab_size))
model.add(Activation('softmax'))
model.compile(loss=sparse_categorical_crossentropy,
              optimizer=Adam(learning_rate),
              metrics=['accuracy'])
print(model.summary())

epochs =1000

model.fit(tmp_x, preproc_answer_sentences, batch_size=128, epochs=epochs, validation_split=0.2)



# Different try Seq2Seq


In [ ]:
# New code 
question_context = Input(shape=(max_encoder_seq_length,), dtype='int32')
encoder_Embedding = Embedding(output_dim=input_dimenstions, input_dim=num_encoder_tokens,weights=[encoder_embedding_matrix], trainable= False)#, input_length=max_encoder_seq_length)
LSTM_encoder = LSTM(hidden_dim, kernel_initializer= 'lecun_uniform', return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c]

answer_context = Input(shape=(max_decoder_seq_length,), dtype='int32')
LSTM_decoder = LSTM(hidden_dim, kernel_initializer= 'lecun_uniform', return_state=True)
decoder_Embedding = Embedding(output_dim=input_dimenstions, input_dim=num_decoder_tokens, weights=[encoder_embedding_matrix], trainable= False, input_length=max_decoder_seq_length)

#Shared_Embedding = Embedding(output_dim=input_dimenstions, input_dim=dictionary_size, weights=[embedding_matrix], input_length=maxlen_input, name='Shared')

word_embedding_context = encoder_Embedding(input_context)
context_embedding = LSTM_encoder(word_embedding_context)

word_embedding_answer = decoder_Embedding(input_answer)
answer_embedding = LSTM_decoder(word_embedding_answer)

merge_layer = merge([context_embedding, answer_embedding], mode='concat', concat_axis=1)
out = Dense(input_dimenstions, activation="relu")(merge_layer)
out = Dense(input_dimenstions, activation="softmax")(out)

model = Model(input=[input_context, input_answer], output = [out])
ad = Adam(lr=0.00005) 
model.compile(loss='categorical_crossentropy', optimizer=ad)

model.fit([encoder_input_data, decoder_input_data], decoder_target_data, batch_size=batch_size, epochs=1)

In [ ]:
# Inputs 
#tmp_x, preproc_answer_sentences

# Old model

In [53]:

def model_final6(input_shape, output_sequence_length, question_vocab_size, answer_vocab_size):
    """
    Build and train a model that incorporates embedding, encoder-decoder, and bidirectional RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param question_vocab_size: Number of unique English words in the dataset
    :param answer_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Implement
    print(input_shape[1:])
    print(output_sequence_length)
    
    learning_rate = 0.01
    hidden_dim = 256
    embedding_vector_length = 100
    
    #Variation - 3 Hidden LSTM Layer and 2 Dense ouput layer with higher Dropout Rate
    
    model = Sequential()
    model.add(Embedding(question_vocab_size, embedding_vector_length, weights=encoder_embedding_matrix,trainable= False,
                        input_shape= input_shape[1:]))
    model.add(Bidirectional(LSTM(hidden_dim, return_sequences=True)))
    model.add(Dropout(0.2))
    model.add(Bidirectional(LSTM(hidden_dim, return_sequences=True)))
    model.add(Dropout(0.2))
    model.add(Bidirectional(LSTM(hidden_dim, return_sequences=True)))
    model.add(Dropout(0.2))
    model.add(Dense(answer_vocab_size))
    model.add(Activation('sigmoid'))
    model.add(Dropout(0.5))
    model.add(Dense(answer_vocab_size))
    model.add(Activation('softmax'))
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    print(model.summary())
    return model


#tests.test_model_final(model_final)


# Reshaping the input to work with a basic RNN
tmp_x = pad(preproc_question_sentences, preproc_answer_sentences.shape[1])
tmp_x = tmp_x.reshape((-1, preproc_answer_sentences.shape[-2]))

# Train the neural network
final_rnn_model6 = model_final6(
    tmp_x.shape,
    preproc_answer_sentences.shape[1],
    len(question_tokenizer.word_index)+1,
    len(answer_tokenizer.word_index)+1)
final_rnn_model6.fit(tmp_x, preproc_answer_sentences, batch_size=128, epochs=500, validation_split=0.2)

final_rnn_model6.save('./saved_models/final_rnn_model6_embed.hdf5')

# Print prediction(s)
print(logits_to_text(final_rnn_model6.predict(tmp_x[:1])[0], answer_tokenizer))

print('Final Model Loaded')

(15,)
15


ValueError: You called `set_weights(weights)` on layer "embedding_6" with a  weight list of length 2707, but the layer was expecting 1 weights. Provided weights: [[ 0.          0.          0.         ...,  0.    ...

In [44]:
print(logits_to_text(loaded_model.predict(tmp_x[:1])[0], answer_tokenizer))

False


In [29]:
model.summary()

NameError: name 'model' is not defined

In [40]:
print(logits_to_text(model.predict(tmp_x[25:30])[0], answer_tokenizer))


It <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>


In [ ]:
#preproc_answer_sentences[6]

len(answer_tokenizer.word_index)+1
#print(logits_to_text(final_rnn_model6.predict(tmp_x[20]), answer_tokenizer))


## Prediction (IMPLEMENTATION)

In [52]:
def preprocess_test(raw_word, name, question_tokenizer):
    l1 = ['won’t','won\'t','wouldn’t','wouldn\'t','’m', '’re', '’ve', '’ll', '’s','’d', 'n’t', '\'m', '\'re', '\'ve', '\'ll', '\'s', '\'d', 'can\'t', 'n\'t', 'B: ', 'A: ', ',', ';', '.', '?', '!', ':', '. ?', ',   .', '. ,', 'EOS', 'BOS', 'eos', 'bos']
    l2 = ['will not','will not','would not','would not',' am', ' are', ' have', ' will', ' is', ' had', ' not', ' am', ' are', ' have', ' will', ' is', ' had', 'can not', ' not', '', '', ' ,', ' ;', ' .', ' ?', ' !', ' :', '? ', '.', ',', '', '', '', '']
#   l3 = ['-', '_', ' *', ' /', '* ', '/ ', '\"', ' \\"', '\\ ', '--', '...', '. . .']
#    l4 = ['jeffrey','fred','benjamin','paula','walter','rachel','andy','helen','harrington','kathy','ronnie','carl','annie','cole','ike','milo','cole','rick','johnny','loretta','cornelius','claire','romeo','casey','johnson','rudy','stanzi','cosgrove','wolfi','kevin','paulie','cindy','paulie','enzo','mikey','i\97','davis','jeffrey','norman','johnson','dolores','tom','brian','bruce','john','laurie','stella','dignan','elaine','jack','christ','george','frank','mary','amon','david','tom','joe','paul','sam','charlie','bob','marry','walter','james','jimmy','michael','rose','jim','peter','nick','eddie','johnny','jake','ted','mike','billy','louis','ed','jerry','alex','charles','tommy','bobby','betty','sid','dave','jeffrey','jeff','marty','richard','otis','gale','fred','bill','jones','smith','mickey']    

    raw_word = raw_word.lower()
    raw_word = raw_word.replace(', ' + name_of_computer, '')
    raw_word = raw_word.replace(name_of_computer + ' ,', '')

    for j, term in enumerate(l1):
        raw_word = raw_word.replace(term,l2[j])
        
 #   for term in l3:
 #       raw_word = raw_word.replace(term,' ')
       
    for j in range(30):
        raw_word = raw_word.replace('. .', '')
        raw_word = raw_word.replace('.  .', '')
        raw_word = raw_word.replace('..', '')
       
    for j in range(5):
        raw_word = raw_word.replace('  ', ' ')
        
    #if raw_word[-1] !=  '!' and raw_word[-1] != '?' and raw_word[-1] != '.' and raw_word[-2:] !=  '! ' and raw_word[-2:] != '? ' and raw_word[-2:] != '. ':
    #    raw_word = raw_word + ' .'
    
    #if raw_word == ' !' or raw_word == ' ?' or raw_word == ' .' or raw_word == ' ! ' or raw_word == ' ? ' or raw_word == ' . ':
    #    raw_word = 'what ?'
       
    
    #if raw_word == '  .' or raw_word == ' .' or raw_word == '  . ':
    #    raw_word = 'i do not want to talk about it .'
    
    
    #raw_new_string = []
    
    #for word in nltk.tokenize.word_tokenize(raw_word):
    #    if word not in question_tokenizer.word_index.keys():
    #        word = name
    #        raw_new_string.append(word)
    #    else:
    #        raw_new_string.append(word)
            
    
    
            #for dict_word in question_tokenizer.word_index.keys():
            #    nlp_words = nlp(word)
            #    dict_tokens = nlp(dict_word)
            #    nlp_words.similarity(dict_tokens)
    
    
    return raw_word


spacy.tokens.doc.Doc

## Submission
When you are ready to submit your project, do the following steps:
1. Ensure you pass all points on the [rubric](https://review.udacity.com/#!/rubrics/1004/view).
2. Submit the following in a zip file.
  - `helper.py`
  - `machine_translation.ipynb`
  - `machine_translation.html`
    - You can export the notebook by navigating to **File -> Download as -> HTML (.html)**.